<span style="font-width:bold; font-size: 3rem; color:#333;">- Part 02: Daily Feature Pipeline for Air Quality (aqicn.org) and weather (openmeteo)</span>

## 🗒️ This notebook is divided into the following sections:
1. Download and Parse Data
2. Feature Group Insertion


__This notebook should be scheduled to run daily__

In the book, we use a GitHub Action stored here:
[.github/workflows/air-quality-daily.yml](https://github.com/featurestorebook/mlfs-book/blob/main/.github/workflows/air-quality-daily.yml)

However, you are free to use any Python Orchestration tool to schedule this program to run daily.

### <span style='color:#ff5f27'> 📝 Imports

In [7]:
import concurrent.futures
import datetime
import time
import requests
import pandas as pd
import hopsworks
from functions import util
import json
import os
import warnings
warnings.filterwarnings("ignore")

from dotenv import load_dotenv
import os

env_path = '../../data/.env'
load_dotenv(dotenv_path=env_path)

HOPSWORKS_API_KEY = os.getenv("HOPSWORKS_API_KEY")

## <span style='color:#ff5f27'> 🌍 Get the Sensor URL, Country, City, Street names from Hopsworks </span>

__Update the values in the cell below.__

__These should be the same values as in notebook 1 - the feature backfill notebook__


In [8]:
project = hopsworks.login(project='AMomozZz')
fs = project.get_feature_store() 
secrets = util.secrets_api(project.name)

# This line will fail if you have not registered the AQI_API_KEY as a secret in Hopsworks
AQI_API_KEY = secrets.get_secret("AQI_API_KEY").value
location_str = secrets.get_secret("SENSOR_LOCATION_JSON").value
location = json.loads(location_str)

country=location['country']
city=location['city']
street=location['street']
aqicn_url=location['aqicn_url']
latitude=location['latitude']
longitude=location['longitude']

today = datetime.date.today()
yesterday = today - datetime.timedelta(days=1)

location_str, today

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1163416
Connected. Call `.close()` to terminate connection gracefully.
Connected. Call `.close()` to terminate connection gracefully.


('{"country": "china", "city": "shanghai", "street": "jinganjiancezhan", "aqicn_url": "https://api.waqi.info/feed/@486", "latitude": 31.23, "longitude": 121.47}',
 datetime.date(2024, 11, 18))

### <span style="color:#ff5f27;"> 🔮 Get references to the Feature Groups </span>

In [9]:
# Retrieve feature groups
air_quality_fg = fs.get_feature_group(
    name='new_aq',
    version=1,
)
weather_fg = fs.get_feature_group(
    name='new_w',
    version=1,
)

## <span style='color:#ff5f27'> 🌫 Retrieve Today's Air Quality data (PM2.5) from the AQI API</span>


In [13]:
import requests
import pandas as pd

aq_today_df = util.get_pm25(aqicn_url, country, city, street, today, AQI_API_KEY)
display(aq_today_df)
aq_today_df.info()

,pm25,country,city,street,date,url
0,144.0,china,shanghai,jinganjiancezhan,2024-11-12,https://api.waqi.info/feed/@486


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   pm25     1 non-null      float32       
 1   country  1 non-null      object        
 2   city     1 non-null      object        
 3   street   1 non-null      object        
 4   date     1 non-null      datetime64[ns]
 5   url      1 non-null      object        
dtypes: datetime64[ns](1), float32(1), object(4)
memory usage: 176.0+ bytes


## <span style='color:#ff5f27'> 🌦 Get Weather Forecast data</span>

In [14]:
hourly_df = util.get_hourly_weather_forecast(city, latitude, longitude)
hourly_df = hourly_df.set_index('date')

# We will only make 1 daily prediction, so we will replace the hourly forecasts with a single daily forecast
# We only want the daily weather data, so only get weather at 12:00
daily_df = hourly_df.between_time('11:59', '12:01')
daily_df = daily_df.reset_index()
daily_df['date'] = pd.to_datetime(daily_df['date']).dt.date
daily_df['date'] = pd.to_datetime(daily_df['date'])
daily_df['city'] = city

df_tail21 = air_quality_fg.read().sort_values(by=['date'], ascending=True).tail(21)

for i in range(1, 21):
    df_temp = df_tail21.copy()
    df_temp['date'] = pd.to_datetime(df_temp['date']) + pd.Timedelta(days=i)
    df_temp['date'] = pd.to_datetime(df_temp['date']).dt.tz_localize(None)
    daily_df = pd.merge(daily_df, df_temp[['date','pm25']], how='left', on="date")
    daily_df = daily_df.rename(columns={"pm25": f"pm25_{i}"})

# today's pm25 value
pm25_value = aq_today_df.iloc[0]['pm25']
today_date = aq_today_df.iloc[0]['date']
# add pm25 value from today
for i in range(1, 21):
    date = today_date + pd.Timedelta(days=i)
    condition = (daily_df['date'] == date)
    daily_df.loc[condition, f'pm25_{i}'] = pm25_value

display(daily_df)
daily_df.info()

Coordinates 31.25°N 121.5°E
Elevation 3.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.27s) 


,date,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant,city,pm25_1,pm25_2,pm25_3,pm25_4,...,pm25_11,pm25_12,pm25_13,pm25_14,pm25_15,pm25_16,pm25_17,pm25_18,pm25_19,pm25_20
0,2024-11-12,16.750000,0.0,7.145796,49.085716,shanghai,132.0,56.0,46.0,55.0,...,20.0,49.0,54.0,49.0,115.0,34.0,23.0,47.0,46.0,95.0
1,2024-11-13,17.400000,0.0,7.862518,105.945465,shanghai,144.0,132.0,56.0,46.0,...,103.0,20.0,49.0,54.0,49.0,115.0,34.0,23.0,47.0,46.0
2,2024-11-14,20.500000,0.0,7.421590,129.093857,shanghai,NaN,144.0,132.0,56.0,...,140.0,103.0,20.0,49.0,54.0,49.0,115.0,34.0,23.0,47.0
3,2024-11-15,20.299999,0.1,7.771331,103.392517,shanghai,NaN,NaN,144.0,132.0,...,124.0,140.0,103.0,20.0,49.0,54.0,49.0,115.0,34.0,23.0
4,2024-11-16,20.500000,0.0,11.966953,6.911131,shanghai,NaN,NaN,NaN,144.0,...,45.0,124.0,140.0,103.0,20.0,49.0,54.0,49.0,115.0,34.0
5,2024-11-17,13.300000,0.0,22.881956,12.724360,shanghai,NaN,NaN,NaN,NaN,...,35.0,45.0,124.0,140.0,103.0,20.0,49.0,54.0,49.0,115.0
6,2024-11-18,11.600000,0.0,22.539564,26.564985,shanghai,NaN,NaN,NaN,NaN,...,35.0,35.0,45.0,124.0,140.0,103.0,20.0,49.0,54.0,49.0
7,2024-11-19,10.950000,0.0,12.261158,3.366400,shanghai,NaN,NaN,NaN,NaN,...,55.0,35.0,35.0,45.0,124.0,140.0,103.0,20.0,49.0,54.0
8,2024-11-20,12.050000,0.0,6.696387,53.746078,shanghai,NaN,NaN,NaN,NaN,...,46.0,55.0,35.0,35.0,45.0,124.0,140.0,103.0,20.0,49.0
9,2024-11-21,13.000000,0.0,6.989935,348.111359,shanghai,NaN,NaN,NaN,NaN,...,56.0,46.0,55.0,35.0,35.0,45.0,124.0,140.0,103.0,20.0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 26 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   date                         10 non-null     datetime64[ns]
 1   temperature_2m_mean          10 non-null     float32       
 2   precipitation_sum            10 non-null     float32       
 3   wind_speed_10m_max           10 non-null     float32       
 4   wind_direction_10m_dominant  10 non-null     float32       
 5   city                         10 non-null     object        
 6   pm25_1                       2 non-null      float32       
 7   pm25_2                       3 non-null      float32       
 8   pm25_3                       4 non-null      float32       
 9   pm25_4                       5 non-null      float32       
 10  pm25_5                       6 non-null      float32       
 11  pm25_6                       7 non-null      flo

## <span style="color:#ff5f27;"> Display before upload</span>

In [20]:
# temp = weather_fg.read().sort_values(by=['date'], ascending=True).tail(11)
# for i in range(1, 12):
#     temp.at[temp.index[i - 1], f'pm25_{i}'] = 25
# temp

# weather_fg.insert(temp, write_options={"wait_for_job": True})
display(air_quality_fg.read().sort_values(by=['date'], ascending=True).tail(5))
display(weather_fg.read().sort_values(by=['date'], ascending=True).tail(15))

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (3.87s) 


,date,pm25,country,city,street,url
3955,2024-11-14 00:00:00+00:00,68.0,china,shanghai,jinganjiancezhan,https://api.waqi.info/feed/@486
3956,2024-11-15 00:00:00+00:00,156.0,china,shanghai,jinganjiancezhan,https://api.waqi.info/feed/@486
3959,2024-11-16 00:00:00+00:00,25.0,china,shanghai,jinganjiancezhan,https://api.waqi.info/feed/@486
3957,2024-11-17 00:00:00+00:00,13.0,china,shanghai,jinganjiancezhan,https://api.waqi.info/feed/@486
3958,2024-11-18 00:00:00+00:00,34.0,china,shanghai,jinganjiancezhan,https://api.waqi.info/feed/@486


Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.44s) 


,date,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant,city,pm25_1,pm25_2,pm25_3,pm25_4,...,pm25_11,pm25_12,pm25_13,pm25_14,pm25_15,pm25_16,pm25_17,pm25_18,pm25_19,pm25_20
3828,2024-11-13 00:00:00+00:00,17.450001,0.0,7.968939,108.435043,shanghai,144.0,132.0,56.0,46.0,...,103.0,20.0,49.0,54.0,49.0,115.0,34.0,23.0,47.0,46.0
3822,2024-11-14 00:00:00+00:00,19.150000,1.5,7.787991,146.309906,shanghai,76.0,144.0,132.0,56.0,...,140.0,103.0,20.0,49.0,54.0,49.0,115.0,34.0,23.0,47.0
3819,2024-11-15 00:00:00+00:00,19.600000,0.0,6.638072,40.601215,shanghai,68.0,76.0,144.0,132.0,...,124.0,140.0,103.0,20.0,49.0,54.0,49.0,115.0,34.0,23.0
3823,2024-11-16 00:00:00+00:00,17.850000,0.0,10.829958,21.447809,shanghai,156.0,68.0,76.0,144.0,...,45.0,124.0,140.0,103.0,20.0,49.0,54.0,49.0,115.0,34.0
3825,2024-11-17 00:00:00+00:00,14.100000,0.0,14.458382,18.886187,shanghai,25.0,156.0,68.0,76.0,...,35.0,45.0,124.0,140.0,103.0,20.0,49.0,54.0,49.0,115.0
3820,2024-11-18 00:00:00+00:00,12.900000,0.0,11.885453,35.134277,shanghai,13.0,25.0,156.0,68.0,...,35.0,35.0,45.0,124.0,140.0,103.0,20.0,49.0,54.0,49.0
3827,2024-11-19 00:00:00+00:00,12.300000,0.0,6.489992,340.559875,shanghai,34.0,13.0,25.0,156.0,...,55.0,35.0,35.0,45.0,124.0,140.0,103.0,20.0,49.0,54.0
3821,2024-11-20 00:00:00+00:00,12.500000,0.0,6.569383,9.462261,shanghai,NaN,34.0,13.0,25.0,...,46.0,55.0,35.0,35.0,45.0,124.0,140.0,103.0,20.0,49.0
3824,2024-11-21 00:00:00+00:00,12.300000,0.0,5.804825,352.875061,shanghai,NaN,NaN,34.0,13.0,...,56.0,46.0,55.0,35.0,35.0,45.0,124.0,140.0,103.0,20.0
3829,2024-11-22 00:00:00+00:00,12.650000,0.0,9.449572,17.744766,shanghai,NaN,NaN,NaN,34.0,...,132.0,56.0,46.0,55.0,35.0,35.0,45.0,124.0,140.0,103.0


## <span style="color:#ff5f27;">⬆️ Uploading new data to the Feature Store</span>

We use multi-threaded methods start two INSERT tasks at the same time. Wait until the two tasks are over before continuing.

In [16]:
def insert_data(feature_group, data):
    return feature_group.insert(data, write_options={"wait_for_job": True})

with concurrent.futures.ThreadPoolExecutor() as executor:
    future1 = executor.submit(insert_data, weather_fg, daily_df)
    future2 = executor.submit(insert_data, air_quality_fg, aq_today_df)

    concurrent.futures.wait([future1, future2])


2024-11-12 19:27:51,972 INFO: 	22 expectation(s) included in expectation_suite.
Validation succeeded.
2024-11-12 19:27:52,251 INFO: 	1 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1163416/fs/1154119/fg/1345972
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1163416/fs/1154119/fg/1345973


Uploading Dataframe: 0.00% |          | Rows 0/1 | Elapsed Time: 00:00 | Remaining Time: ?

Uploading Dataframe: 0.00% |          | Rows 0/10 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: new_aq_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/1163416/jobs/named/new_aq_1_offline_fg_materialization/executions
Launching job: new_w_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/1163416/jobs/named/new_w_1_offline_fg_materialization/executions


In [17]:
display(air_quality_fg.read().sort_values(by=['date'], ascending=True).tail(5))
display(weather_fg.read().sort_values(by=['date'], ascending=True).tail(15))

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.33s) 


,date,pm25,country,city,street,url
3121,2024-11-08 00:00:00+00:00,55.0,china,shanghai,jinganjiancezhan,https://api.waqi.info/feed/@486
2555,2024-11-09 00:00:00+00:00,46.0,china,shanghai,jinganjiancezhan,https://api.waqi.info/feed/@486
3039,2024-11-10 00:00:00+00:00,56.0,china,shanghai,jinganjiancezhan,https://api.waqi.info/feed/@486
3952,2024-11-11 00:00:00+00:00,132.0,china,shanghai,jinganjiancezhan,https://api.waqi.info/feed/@486
3953,2024-11-12 00:00:00+00:00,144.0,china,shanghai,jinganjiancezhan,https://api.waqi.info/feed/@486


Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.93s) 


,date,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant,city,pm25_1,pm25_2,pm25_3,pm25_4,...,pm25_11,pm25_12,pm25_13,pm25_14,pm25_15,pm25_16,pm25_17,pm25_18,pm25_19,pm25_20
2444,2024-11-07 00:00:00+00:00,14.234082,0.0,14.450535,34.656887,shanghai,35.0,45.0,124.0,140.0,...,34.0,23.0,47.0,46.0,95.0,104.0,82.0,48.0,52.0,73.0
2238,2024-11-08 00:00:00+00:00,17.711164,0.0,18.275984,44.213470,shanghai,35.0,35.0,45.0,124.0,...,115.0,34.0,23.0,47.0,46.0,95.0,104.0,82.0,48.0,52.0
1079,2024-11-09 00:00:00+00:00,18.850750,0.5,21.862406,43.717545,shanghai,55.0,35.0,35.0,45.0,...,49.0,115.0,34.0,23.0,47.0,46.0,95.0,104.0,82.0,48.0
957,2024-11-10 00:00:00+00:00,16.086168,1.2,18.553974,0.802675,shanghai,46.0,55.0,35.0,35.0,...,54.0,49.0,115.0,34.0,23.0,47.0,46.0,95.0,104.0,82.0
3170,2024-11-11 00:00:00+00:00,15.100000,0.0,8.161764,41.423569,shanghai,56.0,46.0,55.0,35.0,...,49.0,54.0,49.0,115.0,34.0,23.0,47.0,46.0,95.0,104.0
3826,2024-11-12 00:00:00+00:00,16.750000,0.0,7.145796,49.085716,shanghai,132.0,56.0,46.0,55.0,...,20.0,49.0,54.0,49.0,115.0,34.0,23.0,47.0,46.0,95.0
3828,2024-11-13 00:00:00+00:00,17.400000,0.0,7.862518,105.945465,shanghai,144.0,132.0,56.0,46.0,...,103.0,20.0,49.0,54.0,49.0,115.0,34.0,23.0,47.0,46.0
3822,2024-11-14 00:00:00+00:00,20.500000,0.0,7.421590,129.093857,shanghai,NaN,144.0,132.0,56.0,...,140.0,103.0,20.0,49.0,54.0,49.0,115.0,34.0,23.0,47.0
3819,2024-11-15 00:00:00+00:00,20.299999,0.1,7.771331,103.392517,shanghai,NaN,NaN,144.0,132.0,...,124.0,140.0,103.0,20.0,49.0,54.0,49.0,115.0,34.0,23.0
3823,2024-11-16 00:00:00+00:00,20.500000,0.0,11.966953,6.911131,shanghai,NaN,NaN,NaN,144.0,...,45.0,124.0,140.0,103.0,20.0,49.0,54.0,49.0,115.0,34.0


## <span style="color:#ff5f27;">⏭️ **Next:** Part 03: Training Pipeline
 </span> 

In the following notebook you will read from a feature group and create training dataset within the feature store
